In [7]:
import tdmclient.notebook
await tdmclient.notebook.start()


In [8]:
import time
import sys
import cv2
import pygame as pg
import numpy as np

import vision
import kalman

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:
vid = cv2.VideoCapture(1)


In [10]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500):
    """
    Sets the motor speeds of the Thymio
    param l_speed: left motor speed
    param r_speed: right motor speed
    """
    global motor_left_target, motor_right_target

    motor_left_target = l_speed
    motor_right_target = r_speed

In [11]:
@tdmclient.notebook.sync_var
def read_motors():
    """
    Read the motor speeds of the Thymio
    """
    global motor_left_speed, motor_right_speed

    return [motor_left_speed, motor_right_speed]

In [12]:
%%run_python
nf_leds_prox_h(0,0,0,0,0,0,0,0)
nf_leds_prox_v(0,0)
nf_leds_rc(0)
nf_leds_sound(0)
nf_leds_temperature(0,0)
nf_leds_top(0,0,0)
nf_leds_bottom_left(0,0,0)
nf_leds_bottom_right(0,0,0)
nf_leds_buttons(30,0,0,0)
nf_leds_top(0,255,0)


In [13]:

# Set simulation parameters
Ts = 0.01

# Initialize kalman filter
mus_pos = []
covs_pos = []
mus_att = []
covs_att = []

kf_pos = kalman.Kalman(_acc_variance=0.5, _type="pose", videoFeed=vid)
kf_att = kalman.Kalman(_acc_variance=0.5, _type="orientation", videoFeed=vid)

print(kf_pos.mean)
print(kf_att.mean*180/np.pi)
print("---------------------")

# Initialize pygame
pg.init()

h = 841
w = 1189
scr = pg.display.set_mode((w, h))

thymio = pg.image.load("thymio.png")
thymio = pg.transform.scale(thymio, (50, 50))


# main loop
step = 0
running = True
simulate = False
while running:
    for event in pg.event.get():
        if event.type == pg.QUIT:
            pg.quit()
            sys.exit()
        elif event.type == pg.KEYDOWN:
            if event.key == pg.K_RETURN :
                simulate = not simulate
    # Set up environment
    scr.fill((0, 0, 0))
    pg.draw.line(scr, (255, 255, 255), (0, int(h/2)), (w, int(h/2)))
    pg.draw.line(scr, (255, 255, 255), (int(w/2), 0), (int(w/2), h))

    # Start thymio movement
    if simulate:
        step+=1

        # Save expected values and covariance matrices
        mus_pos.append(kf_pos.mean)
        covs_pos.append(kf_pos.cov)
        mus_att.append(kf_att.mean)
        covs_att.append(kf_att.cov)

        # Read sensor data
        video_odometry = vision.fetchOdoMeters(vid)
        if type(video_odometry) != bool:
            pos = video_odometry[0:2]
            angle = np.array([video_odometry[2]])
        motor_speeds = read_motors()
        speed = (sum(motor_speeds)/2)*0.0005 # magnitude of velocity [m/s]
        angular_speed = (motor_speeds[1]-motor_speeds[0])*0.0005/0.094 # magnitude of angular velocity [rad/s]
        velocity = [speed*np.cos(mus_att[-1][0]), speed*np.sin(mus_att[-1][0])]

        # Estimate pose
        kf_pos.predict(dt=Ts)
        kf_pos.update(meas_value=velocity,
               meas_variance=kalman.meas_variance_vel,
               C=np.array([[0, 0, 1, 0], [0, 0, 0, 1]]))
        if type(video_odometry) != bool:
            kf_pos.update(meas_value=list(pos),
                meas_variance=kalman.meas_variance_pos,
                C=np.array([[1, 0, 0, 0], [0, 1, 0, 0]]))

        # Estimate orientation
        kf_att.predict(dt=Ts)
        kf_att.update(meas_value=angular_speed,
               meas_variance=kalman.meas_variance_omega,
               C=np.array([0, 1]))
        if type(video_odometry) != bool:
            kf_att.update(meas_value=angle[0],
                meas_variance=kalman.meas_variance_att,
                C=np.array([1, 0]))

        # Update thymio position
        kalman.move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        kalman.drawLine([mu[0] for mu in mus_pos], [mu[1] for mu in mus_pos], w, h, scr)

        # Motion control



        # Flow control
        motors(100, 200)
    else:
        kalman.move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        kalman.drawLine([mu[0] for mu in mus_pos], [mu[1] for mu in mus_pos], w, h, scr)
        motors(0, 0)

    # Update screen
    pg.display.flip()
    time.sleep(Ts)

motors(0, 0)
vid.release()

Odometry obtained [350 384] 178.69148454943368
Odometry obtained [350 384] 179.0486133224723
[0.61322917 0.6728     0.         0.        ]
[179.04846209   0.        ]
---------------------
Odometry obtained [350 383] 179.64766537533112
Odometry obtained [350 383] 176.9957574757291
Odometry obtained [350 383] 176.6360113051083
Odometry obtained [350 383] 178.79015059333045
Odometry obtained [350 383] 179.8433493990465
Odometry obtained [348 383] 179.1556402674835
Odometry obtained [347 384] -179.30927976180456
Odometry obtained [346 384] -178.3662783541418
Odometry obtained [345 383] -179.3449218226982
Odometry obtained [344 383] -178.43730838463384
Odometry obtained [341 383] -175.71187055513266
Odometry obtained [340 383] -175.21537002043567
Odometry obtained [339 383] -173.04507136216583
Odometry obtained [338 383] -172.68073576150115
Odometry obtained [335 383] -170.75663152630298
Odometry obtained [334 383] -169.4016325651535
Odometry obtained [333 382] -170.38177569339857
Odometry

SystemExit: 

c:\users\timme\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
motors(100, 100)
stop()

NameError: name 'motors' is not defined

In [8]:

motors(0,0)

In [5]:
stop()